<a href="https://colab.research.google.com/github/MaansAndersson/Krylov_solvers/blob/main/Poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
!dpkg --configure -a
!apt --fix-broken install python-pycurl python-apt

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion
!pip install ideep4py
if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3")
    sys.exit()
    
try:
    !python3 -c 'from distutils.sysconfig import get_python_lib; print(get_python_lib(prefix="/usr/local"))'
    from dolfin import *; from mshr import *
except ImportError as e:
    !python3 -c 'from distutils.sysconfig import get_python_lib; print(get_python_lib(prefix="/usr/local"))'

    sys.path.append('/usr/lib/python3/dist-packages/')
    import sys, os; petsc_dir = '/usr/lib/petsc'; petsc_dir = os.path.join(os.getenv('PETSC_DIR') or petsc_dir, 'lib/python3/dist-packages'); petsc_dir in sys.path or sys.path.append(petsc_dir);
    #!apt-get install python3-dolfin
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
    !add-apt-repository -y ppa:fenics-packages/fenics #ppa:fenics-packages/fenics
    #!apt-get update -qq
    !apt install -y fenics #--no-install-recommends 
    !python3 -c 'from distutils.sysconfig import get_python_lib; print(get_python_lib(prefix="/usr/local"))'

    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; # import dolfin.common.plotting as fenicsplot 

import os, sys, shutil
import numpy as np 

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  import cupy as cp
  import scipy as sp
  from scipy.sparse import csr_matrix
  !git clone https://github.com/MaansAndersson/Krylov_solvers.git
  from Krylov_solvers import KLS 
  print(gpu_info)

dolfin_version = dolfin.__version__
print('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pycurl is already the newest version (7.43.0.1-0.2).
python-apt is already the newest version (1.6.5ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
/usr/local/lib/python3/dist-packages
Cloning into 'Krylov_solvers'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.
Mon Feb  1 15:15:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Uti

In [91]:
from timeit import default_timer as timer
import sys, petsc4py
petsc4py.init(sys.argv)
from petsc4py import PETSc
import time

## CREATE PROBLEM ##
mesh = UnitSquareMesh(128,128)
#mesh = UnitSquareMesh(64,64)

pdeg = 6

V = FunctionSpace(mesh, 'CG', pdeg)

f = Expression('40*pi*pi*sin(2*pi*x[0])*sin(4*pi*x[1])', pi = dolfin.pi, degree = pdeg)
bc = DirichletBC(V,Expression(('0'), degree = pdeg ), "on_boundary")
v = TestFunction(V)
u = TrialFunction(V)

a = inner(grad(u),grad(v))*dx
L = inner(v,f)*dx

## Assembled system ##
A, b = assemble_system(a, L, bc)


## Fenics linear solve
#  
u = Function(V)
start = timer()
solve(A, u.vector(), b, "cg")
end = timer()

print('FEniCS cg: ',end - start)
print('  ')


## Fenics Linear Problem
#
u = Function(V)
problem = LinearVariationalProblem(a, L, u, bc)
solver = LinearVariationalSolver(problem)
solver.parameters["linear_solver"]  = "cg";
solver.parameters["krylov_solver"]["relative_tolerance"] = 1e-8
solver.parameters["krylov_solver"]["monitor_convergence"] = True
start = timer()
solver.solve()
end = timer()

print('FEniCS LinearProblem cg: ',end - start)
print('  ')


## Backend PETSc
#
Xf = Function(V)
start = timer()
ksp = PETSc.KSP()
ksp.create()
ksp.setType('cg')
ksp.max_it = 5000
ksp.rtol = 1e-8
mat = as_backend_type(A).mat() 
ai, aj, av = mat.getValuesCSR()
Q = PETSc.Mat().createAIJ(size=(len(b),len(b)), csr=(ai,aj,av))
B = as_backend_type(b)
X = as_backend_type(Xf.vector())
ksp.setOperators(Q)
ksp.solve(B.vec(), X.vec())
end = timer()

print("   iterations: %d residual norm: %g" % (ksp.its, ksp.norm)) 
print("PETSC solver time", end - start)
print('  ')

## KLS TEST
# 
u = Function(V)
start = timer()
u.vector()[:] = KLS.linear_solver(A, u.vector()[:], b, 'CG', 1e-8, 5000, 'True', 'True')
end = timer() 
print("KLS solver time", end - start)
print('  ')

start = timer()
u.vector()[:] = KLS.linear_solver(A, u.vector()[:], b, 'CG', 1e-10, 5000, 'True', 'True')
end = timer() 
print("KLS solver time", end - start)
print('  ')




FEniCS cg:  20.794440106000366
  
FEniCS LinearProblem cg:  89.65602116300033
  
   iterations: 580 residual norm: 3.77707e-09
PETSC solver time 60.36017289500069
  
   Linear solver residual norm;  8.999673587430142e-09 n of iterations:  468
   Linear solver timing 2.9421669999999267
KLS solver time 2.908061946000089
  
   Linear solver residual norm;  9.3176947401804e-11 n of iterations:  1232
   Linear solver timing 6.947114000000056
KLS solver time 6.952374941000016
  
